# Introduction

Notebook para fazer análise de modelos e de resultados de experimentos feitos e armazenados no Neptune

# Import Libraries

In [1]:
import os
import sys
import neptune.new as neptune
import zipfile
import shutil
import numpy as np
import pandas as pd

from datetime import datetime

from tqdm.notebook import tqdm

if '..' not in sys.path:
    sys.path.insert(0, '..')

import config

# Download de Modelos

## Utilitary Functions

In [2]:
def process_gt_names(gt_names):
    gt_names_dict = {'train_validation_test': gt_names.split(':')[-1]}
    train_val_test = gt_names_dict['train_validation_test']
    train_val_test = train_val_test[:-3]
    train_val_test = train_val_test[3:]
    return train_val_test


def download_model(exp_id, is_mtl):
    try:
        neptune_run = None
        print(f'Experiment ID: {exp_id}')

        neptune_run = neptune.init(project='guilhermemg/icao-nets-training-2', 
                                   api_token=config.NEPTUNE_API_TOKEN,
                                   run=exp_id)

        ds = neptune_run['properties/gt_names'].fetch()
        alg = neptune_run['properties/aligned'].fetch()
        req = neptune_run['properties/icao_reqs'].fetch()

        ds = process_gt_names(ds)
        aligned = 'aligned' if bool(alg) == True else 'not_aligned'
        req = req[2:-2]
        
        print(f' Requisite: {req.upper()} | Dataset: {ds.upper()} | Aligned: {aligned}')
        
        if not is_mtl:
            dest_dir = f'prev_trained_models/single_task/{ds}_{aligned}/{req}/{exp_id}'
        else:
            dest_dir = f'prev_trained_models/multi_task/{ds}_{aligned}/multi_reqs/{exp_id}'
        
        print(f'Destination path: {dest_dir}')
        
        if os.path.exists(dest_dir):
            #shutil.rmtree(dest_dir)
            print('Model already downloaded!')
            print('-----------------------------')
            return
            
        os.makedirs(dest_dir)
        
        try:
            success_download = False
            neptune_run['artifacts/trained_model'].download(dest_dir)
            success_download = True

            with zipfile.ZipFile(os.path.join(dest_dir, 'trained_model.zip'), 'r') as zip_ref:
                zip_ref.extractall(dest_dir)
                
            folder_name = [x for x in os.listdir(dest_dir) if '.zip' not in x][0]
            
            print(folder_name)

            os.remove(os.path.join(dest_dir, f'{folder_name}.zip'))
            shutil.move(os.path.join(dest_dir, folder_name, 'variables'), dest_dir)
            shutil.move(os.path.join(dest_dir, folder_name, 'saved_model.pb'), dest_dir)
            shutil.rmtree(os.path.join(dest_dir, folder_name))

            print('.. Folders set')
        except Exception as ex:
            print(f'Experiment {exp_id} did not produce a trained model!')
        finally:
            if not success_download:
                shutil.rmtree(dest_dir)
            print('-----------------------------')
            return
            
    except Exception as e:
        raise e
    finally:
        if neptune_run is not None:
            neptune_run.stop()

## Single Task Networks

In [3]:
df = pd.read_csv('single_task_exps_data/icao-nets-training-2.csv')
df.head()

,Id,Creation Time,Owner,Tags,properties/icao_reqs,properties/aligned,properties/gt_names
0,ICAO-186,2021-06-12 13:21:22,guilhermemg,"[""ground truths"",""vgg16"",""binary_output"",""adam...",['red_eyes'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
1,ICAO-185,2021-06-12 13:18:09,guilhermemg,"[""binary_output"",""ground truths"",""adamax"",""vgg...",['background'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
2,ICAO-184,2021-06-12 13:14:38,guilhermemg,"[""fvc"",""ground truths"",""binary_output"",""adamax...",['hair_eyes'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
3,ICAO-183,2021-06-12 13:11:25,guilhermemg,"[""ground truths"",""pixelation"",""binary_output"",...",['pixelation'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
4,ICAO-182,2021-06-12 13:08:26,guilhermemg,"[""binary_output"",""ground truths"",""vgg16"",""wash...",['washed_out'],0.0,"{'train_validation': [], 'test': [], 'train_va..."


In [4]:
df = pd.read_csv('single_task_exps_data/icao-nets-training-2.csv')

assert df.shape[0] == 23 # 23 REQUISITES!!

for exp_id in df.Id.values:
    download_model(exp_id)            

Experiment ID: ICAO-186
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-186
 Requisite: RED_EYES | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/red_eyes/ICAO-186
Model already downloaded!
-----------------------------
Experiment ID: ICAO-185
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-185
 Requisite: BACKGROUND | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/background/ICAO-185
Model already downloaded!
-----------------------------
Experiment ID: ICAO-184
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-184
 Requisite: HAIR_EYES | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/hair_eyes/ICAO-184
Model already downloaded!
-----------------------------
Experiment ID: ICAO-183
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-183
 Requisite: PIXELATION | Dataset: FVC

## Multi Task Networks - Handcrafted

In [3]:
exp_ids = ['ICAO-199','ICAO-192']

for exp_id in exp_ids:
    download_model(exp_id, is_mtl=True)            

Experiment ID: ICAO-199
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-199
 Requisite: MOUTH', 'ROTATION', 'L_AWAY', 'EYES_CLOSED', 'CLOSE', 'HAT', 'DARK_GLASSES', 'FRAMES_HEAVY', 'FRAME_EYES', 'FLASH_LENSES', 'VEIL', 'REFLECTION', 'LIGHT', 'SH_FACE', 'SH_HEAD', 'BLURRED', 'INK_MARK', 'SKIN_TONE', 'WASHED_OUT', 'PIXELATION', 'HAIR_EYES', 'BACKGROUND', 'RED_EYES | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/multi_task/fvc_not_aligned/multi_reqs/ICAO-199
trained_model
.. Folders set
-----------------------------
Experiment ID: ICAO-192
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-192
 Requisite: MOUTH', 'ROTATION', 'L_AWAY', 'EYES_CLOSED', 'CLOSE', 'HAT', 'DARK_GLASSES', 'FRAMES_HEAVY', 'FRAME_EYES', 'FLASH_LENSES', 'VEIL', 'REFLECTION', 'LIGHT', 'SH_FACE', 'SH_HEAD', 'BLURRED', 'INK_MARK', 'SKIN_TONE', 'WASHED_OUT', 'PIXELATION', 'HAIR_EYES', 'BACKGROUND', 'RED_EYES | Dataset: FVC | Aligned: not_aligned
Destination path: prev